In [ ]:
import mediapipe as mp
import cv2
import os
from os.path import join, isdir
import pandas as pd
import numpy as np
from IPython.display import clear_output 

In [ ]:
# https://www.kaggle.com/datasets/kapillondhe/american-sign-language


ROOT = "/Users/vaisakh/Desktop/last-research/ASL_Dataset/ASL_Train"
EXT = ".jpg"
CSV_PATH = "/Users/vaisakh/research_final_codebase/dataset.csv"

In [ ]:
img_paths = []
for label in os.listdir(ROOT):
    if not isdir(join(ROOT, label)): continue;
    for image in os.listdir(join(ROOT, label)):
        img_path = join(ROOT, label, image)
        if not img_path.endswith(EXT): continue
        img_paths.append([label, img_path])

In [ ]:
len(img_paths)

165670

In [ ]:
cols = [
    '0x', '0y', '1x', '1y', '2x', '2y', '3x', '3y', 
    '4x', '4y', '5x', '5y', '6x', '6y', '7x', '7y', 
    '8x', '8y', '9x', '9y', '10x', '10y', '11x', '11y', 
    '12x', '12y', '13x', '13y', '14x', '14y', '15x', 
    '15y', '16x', '16y', '17x', '17y', '18x', '18y', 
    '19x', '19y', '20x', '20y', 'label']

df = pd.DataFrame(columns=cols)

In [ ]:
def pre_process(landmark):
    processed_landmarks = []
    relative_landmarks= []
    for i in landmark[0].landmark:
        processed_landmarks.append([i.x, i.y])

    for i in processed_landmarks:
        relative_landmarks.append([
            i[0]-processed_landmarks[0][0],
            i[1]-processed_landmarks[0][1],
        ])
    
    flatten = []
    for i in relative_landmarks:
        flatten.append(i[0])
        flatten.append(i[1])
    
    maxVal = max(list(map(abs, flatten)))
    def norm(n): return n/maxVal
    normalised = list(map(norm, flatten))

    return normalised

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) 
count= 0
corrupted_files = []

for label, image_path in img_paths:
    count+=1
    clear_output()
    print(count, image_path)

    image = cv2.imread(image_path)
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = hands.process(image)
        if not result.multi_hand_landmarks: continue

        # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # for hand in result.multi_hand_landmarks:
        #     mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
        #                     mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
        #                     mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),)
        # cv2.imshow("wind", image)
        # cv2.waitKey(0)

        preprocessed = pre_process(result.multi_hand_landmarks)
        preprocessed.append(label)
        df.loc[len(df)] = preprocessed
    except:
        corrupted_files.append(image_path)
        print("corrupted:", image_path)

    

    
# cv2.destroyAllWindows()

165670 /home/amma/Downloads/archive/ASL_Dataset/Train/Z/Z2845.jpg


In [ ]:
df.head()

,0x,0y,1x,1y,2x,2y,3x,3y,4x,4y,...,16y,17x,17y,18x,18y,19x,19y,20x,20y,label
0,0.0,0.0,-0.263195,0.168084,-0.461619,0.238870,-0.545770,0.302678,-0.478784,0.437474,...,0.920620,0.421439,-0.428301,0.404070,0.039545,0.358000,0.382578,0.324619,0.636810,M
1,0.0,0.0,-0.263099,0.028681,-0.421139,0.147102,-0.402727,0.327252,-0.330016,0.455576,...,0.951374,0.363423,0.171764,0.366473,0.455060,0.327095,0.647811,0.283990,0.788677,M
2,0.0,0.0,-0.317940,-0.047910,-0.503678,-0.068887,-0.492297,0.091870,-0.366191,0.284903,...,0.946927,0.469347,-0.137859,0.490483,0.247974,0.477984,0.517449,0.466153,0.725181,M
3,0.0,0.0,-0.316733,0.100902,-0.559333,0.143377,-0.624913,0.156550,-0.485635,0.204360,...,0.951110,0.561934,-0.403880,0.483950,0.065134,0.429496,0.391307,0.392375,0.634365,M
4,0.0,0.0,-0.284665,0.085728,-0.453299,0.167248,-0.372762,0.317592,-0.241684,0.489102,...,0.934091,0.505951,-0.375049,0.536691,0.106561,0.464293,0.487019,0.407274,0.751768,M


In [ ]:
df.to_csv(CSV_PATH)
df['label'].value_counts()

F        5970
L        5970
K        5958
G        5955
J        5953
D        5951
H        5929
B        5929
A        5915
Z        5903
C        5903
R        5896
E        5892
X        5890
I        5887
P        5880
S        5836
V        5835
W        5796
O        5751
Q        5741
Y        5667
T        5530
Space    5191
M        5172
U        4371
N        3674
Name: label, dtype: int64

In [ ]:
print(corrupted_files)

[]
